## Simple Gen AI APP using LangChain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
#Langchain Tracking
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACKING_V2"] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")


In [2]:
## Data  Ingestion -- From the Website we need to scrape the data

from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader = WebBaseLoader("https://docs.langchain.com/oss/python/langchain/models")
loader

In [4]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/oss/python/langchain/models', 'title': 'Models - Docs by LangChain', 'language': 'en'}, page_content='Models - Docs by LangChainSkip to main content🚀 Share how you\'re building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationCore componentsModelsLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestAgent Chat UIDeploy with LangSmithDeploymentObservabilityOn this pageBasic usageInitialize a modelKey methodsParametersInvocationInvokeStreamBatchTool call

In [5]:
## Load Data --> Docs --> Divide our text into chunks --> text --> vectors --> vector Embeddings --> vector store db

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [6]:
print(documents)

[Document(metadata={'source': 'https://docs.langchain.com/oss/python/langchain/models', 'title': 'Models - Docs by LangChain', 'language': 'en'}, page_content="Models - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationCore componentsModelsLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestAgent Chat UIDeploy with LangSmithDeploymentObservabilityOn this pageBasic usageInitialize a modelKey methodsParametersInvocationInvokeStreamBatchTool calli

In [7]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

In [8]:
from langchain_community.vectorstores import FAISS

vector_store_db  = FAISS.from_documents(documents,embedding=embeddings)
vector_store_db

In [9]:
query = "LangSmith has two usage limits: total traces and extended"
result = vector_store_db.similarity_search(query)
result[0].page_content

"Models - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationCore componentsModelsLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestAgent Chat UIDeploy with LangSmithDeploymentObservabilityOn this pageBasic usageInitialize a modelKey methodsParametersInvocationInvokeStreamBatchTool callingStructured outputSupported modelsAdvanced topicsModel profilesMultimodalReasoningLocal modelsPrompt cachingServer-side tool useRate limitingBase URL or"

In [10]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model='llama3')

In [11]:
import langchain
print("langchain =", langchain.__version__)

import langchain_core
print("langchain_core =", langchain_core.__version__)

import langchain_community
print("langchain_community =", langchain_community.__version__)


langchain = 1.1.3
langchain_core = 1.1.3
langchain_community = 0.4.1


In [12]:
import pkgutil
import langchain

modules = [m.name for m in pkgutil.iter_modules(langchain.__path__)]

print("chains" in modules, modules)


False ['agents', 'chat_models', 'embeddings', 'messages', 'rate_limiters', 'tools']


In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = ChatPromptTemplate.from_template("""
Answer using ONLY this context:

<context>
{context}
</context>

Question: {input}

Answer:
""")

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {
        "context": vector_store_db.as_retriever() | RunnableLambda(format_docs),
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
)

rag_chain


{
  context: VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000012E7A114100>, search_kwargs={})
           | RunnableLambda(format_docs),
  input: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer using ONLY this context:\n\n<context>\n{context}\n</context>\n\nQuestion: {input}\n\nAnswer:\n'), additional_kwargs={})])
| ChatOllama(model='llama3')

In [24]:
response = rag_chain.invoke("Models can be requested to provide their response in a format matching a given schema")

In [25]:
print(response.content)

Yes, models can be requested to provide their response in a format matching a given schema. This is demonstrated in the example:

```
model_with_structure = model.with_structured_output(
    json_schema,
    method="json_schema",
)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # {'title': 'Inception', 'year': 2010, ...}
```

Here, the `with_structured_output` method is used to specify a JSON schema for the response. The resulting response will conform to this schema.
